In [1]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import time
import random
import os

import gensim

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
import string
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install torchtext==0.4

     |████████████████████████████████| 61kB 2.2MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [0]:
import torch
from torchtext import data, datasets, vocab, utils
import torch.nn as nn
import torch.nn.functional as F
import torchsummary
import torch.optim as optim

In [0]:
PATH = '/content/drive/My Drive/Colab Notebooks/Fake_News_Detection/dataset/'

In [0]:
df = pd.read_csv(PATH+'data_clean_with_onehot.csv')

In [0]:
train_index = df.query("data_type == 'train'").index
val_index = df.query("data_type == 'val'").index
test_index = df.query("data_type == 'test'").index

In [0]:
y_train_multi = df.loc[train_index, 'label']
y_val_multi = df.loc[val_index, 'label']
y_test_multi = df.loc[test_index, 'label']

y_train_binary = df.loc[train_index, 'is_fake']
y_val_binary = df.loc[val_index, 'is_fake']
y_test_binary = df.loc[test_index, 'is_fake']

In [0]:
train_sentences = df.loc[train_index].statement.values
val_sentences = df.loc[val_index].statement.values
test_sentences = df.loc[test_index].statement.values

# Functions

In [0]:
def get_metrics(true, predicted, binary=True):
  if binary:
    return pd.DataFrame({"Accuracy": [round(accuracy_score(true, predicted), 4)],
  "Precision": [round(precision_score(true, predicted), 4)],
  "Recall": [round(recall_score(true, predicted), 4)],
  "F1": [round(f1_score(true, predicted), 4)]})
  else:
    return pd.DataFrame({"Accuracy": [round(accuracy_score(true, predicted), 4)],
  "Precision": [round(precision_score(true, predicted, average='macro'), 4)],
  "Recall": [round(recall_score(true, predicted, average='macro'), 4)],
  "F1": [round(f1_score(true, predicted, average='macro'), 4)]})

In [0]:
def text_embed_info(feature):
  vocabulary = {}
  for word in ' '.join(df[feature].values).split():
    if word in vocabulary.keys():
      vocabulary[word] += 1
    else:
      vocabulary[word] = 1
  success = 0
  word_cnt = 0
  unknown_words = []
  for word in vocabulary.keys():
    if word in vocab.keys():
      success += 1
      word_cnt += vocabulary[word]
    else:
      unknown_words.append(word)

  print(f"{feature:13} | Found embeddings for {success*100//len(vocabulary.keys())}% of vocabulary")
  print(f"{feature:13} | Found embeddings for {word_cnt*100//len(' '.join(df[feature].values).split())}% of all text")
  print()

In [0]:
def get_embedding_matrix(feature):
  emb_mean, emb_std = embedding.mean(), embedding.std()

  field = embedding_info[feature]['field']

  word_index = dict(field.vocab.stoi).keys()
  unknown_words = set()
  nb_words = len(word_index)
  embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, EMBED_SIZE))
  for word in word_index:
        index = field.vocab[word]
        try:
            embedding_vector = embeddings_model[word]
            embedding_matrix[index] = embedding_vector
        except KeyError:
            unknown_words.add(word)


  embedding_matrix = torch.from_numpy(embedding_matrix) 
  
  print(f"{feature:14}| Vocabulary size {nb_words}\t|Unknown words {len(unknown_words)}")

  embedding_info[feature]['embedding_matrix'] = embedding_matrix
  embedding_info[feature]['unknown_words'] = unknown_words
  embedding_info[feature]['vocab_size'] = nb_words

def get_one_hot_matrix(feature):
  field = embedding_info[feature]['field']
  nb_words = len(field.vocab)
  embedding_matrix = torch.from_numpy(np.eye(nb_words))

  print(f"{feature:14}| Vocabulary size {nb_words}")

  embedding_info[feature]['embedding_matrix'] = embedding_matrix
  embedding_info[feature]['vocab_size'] = nb_words
  
def set_vectors_torch(feature):
  embedding_info[feature]['field'].vocab.set_vectors(
        
      embedding_info[feature]['field'].vocab.stoi, 
      embedding_info[feature]['embedding_matrix'], 
      embedding_info[feature]['embedding_matrix'].shape[1]
      
)

In [0]:
def seed_torch(seed=1568):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True

In [0]:
def get_features_from_batch(batch_):
  return batch_.is_fake.to(device), batch_.label.to(device), batch_.statement.to(device), \
  batch_.subject.to(device), batch_.speaker.to(device), batch_.speaker_job.to(device), batch_.state.to(device), \
  batch_.party.to(device), batch_.context.to(device), batch_.justification.to(device)

def train_func(train_iter_, target_is_binary=True):

    train_loss = 0
    train_acc = 0
    for i, batch in enumerate(train_iter_):
        optimizer.zero_grad()

        target_binar, target_multi, statement, subject, speaker, speaker_job, state, party, context, justification = get_features_from_batch(batch)
        seed_torch()
        output = model(statement, subject, speaker, speaker_job, state, party, context, justification)
        if target_is_binary:
          loss = loss_func(output, target_binar)
          train_acc += (output.argmax(1) == target_binar).sum().item()
        else:
          loss = loss_func(output, target_multi)
          train_acc += (output.argmax(1) == target_multi).sum().item()

        train_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm(model.parameters(), 1)
        optimizer.step()
        

    # Adjust the learning rate
    scheduler.step()

    return train_loss / (len(train_iter_.dataset) / BATCH_SIZE), train_acc / len(train_iter_.dataset)

def test_func(data_, target_is_binary=True):
    loss = 0
    acc = 0

    for batch in data_:
        target_binar, target_multi, statement, subject, speaker, speaker_job, state, party, context, justification = get_features_from_batch(batch)
        with torch.no_grad():
            output = model(statement, subject, speaker, speaker_job, state, party, context, justification)
            if target_is_binary:
              loss += loss_func(output, target_binar).item()
              acc += (output.argmax(1) == target_binar).sum().item()
            else:
              loss += loss_func(output, target_multi).item()
              acc += (output.argmax(1) == target_multi).sum().item()

    return loss / (len(data_.dataset) / BATCH_SIZE), acc / len(data_.dataset)

# Baseline

##Majority vote model (baseline)

### Multiclass classification

In [0]:
multi_popular = y_train_multi.value_counts().index[0]

In [139]:
get_metrics(y_val_multi, [multi_popular]*len(y_val_multi), binary=False)

,Accuracy,Precision,Recall,F1
0,0.1931,0.0322,0.1667,0.054


In [125]:
get_metrics(y_test_multi, [multi_popular]*len(y_test_multi), binary=False)

,Accuracy,Precision,Recall,F1
0,0.2092,0.0349,0.1667,0.0577


### Binary Classification

In [0]:
binary_popular = y_train_binary.value_counts().index[0]

In [140]:
get_metrics(y_val_binary, [binary_popular]*len(y_val_binary), binary=False)

,Accuracy,Precision,Recall,F1
0,0.5202,0.2601,0.5,0.3422


In [127]:
get_metrics(y_test_binary, [binary_popular]*len(y_test_binary), binary=False)

,Accuracy,Precision,Recall,F1
0,0.5635,0.2818,0.5,0.3604


## TF-IDF LogisticRegression

### "Statement" model

In [0]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), sublinear_tf=True, max_df=0.8)

In [0]:
X_train = tfidf.fit_transform(df.loc[train_index, 'statement'])
X_val = tfidf.transform(df.loc[val_index, 'statement'])
X_test = tfidf.transform(df.loc[test_index, 'statement'])

#### Multiclass classification

In [0]:
lr = LogisticRegression(max_iter=500)

In [131]:
lr.fit(X_train, y_train_multi)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [133]:
get_metrics(y_val_multi, lr.predict(X_val), binary=False)

,Accuracy,Precision,Recall,F1
0,0.2609,0.3097,0.235,0.2253


In [132]:
get_metrics(y_test_multi, lr.predict(X_test), binary=False)

,Accuracy,Precision,Recall,F1
0,0.2526,0.3246,0.2234,0.2169


#### Binary Classification 

In [0]:
lr = LogisticRegression(C=2)

In [135]:
lr.fit(X_train, y_train_binary)

LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [141]:
get_metrics(y_val_binary, lr.predict(X_val), binary=False)

,Accuracy,Precision,Recall,F1
0,0.6301,0.641,0.623,0.6146


In [137]:
get_metrics(y_test_binary, lr.predict(X_test))

,Accuracy,Precision,Recall,F1
0,0.614,0.5894,0.3816,0.4632


# GoogleNews word2vec Embeddings

In [15]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

--2020-06-04 14:50:13--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.163.157
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.163.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  70.7MB/s    in 23s     

2020-06-04 14:50:36 (69.3 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [0]:
embeddings_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
vocab = embeddings_model.vocab
embedding = embeddings_model.syn0

In [17]:
text_embed_info('statement')
text_embed_info('subject')
text_embed_info('speaker_job')
text_embed_info('context')
text_embed_info('justification')

statement     | Found embeddings for 88% of vocabulary
statement     | Found embeddings for 90% of all text

subject       | Found embeddings for 98% of vocabulary
subject       | Found embeddings for 98% of all text

speaker_job   | Found embeddings for 92% of vocabulary
speaker_job   | Found embeddings for 82% of all text

context       | Found embeddings for 86% of vocabulary
context       | Found embeddings for 81% of all text

justification | Found embeddings for 80% of vocabulary
justification | Found embeddings for 89% of all text



# Embeddings preprocessin

In [0]:
EMBED_SIZE = 300 

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
STATEMENT = data.Field(sequential=True)
CONTEXT = data.Field(sequential=True)
SUBJECT = data.Field(sequential=True)
SPEAKER_JOB = data.Field(sequential=True)
JUSTIFICATION = data.Field(sequential=True)

SPEAKER = data.Field(sequential=False)
STATE = data.Field(sequential=False)
PARTY = data.Field(sequential=False)

TARGET_BINAR = data.Field(sequential=False, use_vocab=False, dtype=torch.int64)
TARGET_MULTI = data.Field(sequential=False, use_vocab=False, dtype=torch.int64)

data_fields = [('label', TARGET_MULTI),
               ('is_fake', TARGET_BINAR),
               ('statement', STATEMENT),
               ('subject', SUBJECT),
               ('speaker', SPEAKER),
               ('speaker_job', SPEAKER_JOB),
               ('state', STATE),
               ('party', PARTY),
               ('context', CONTEXT),
               ('justification', JUSTIFICATION)
               ]

In [0]:
train = data.TabularDataset(path=PATH+'train_data_clean.csv', format='csv', fields=data_fields, skip_header=True)
val = data.TabularDataset(path=PATH+'val_data_clean.csv', format='csv', fields=data_fields, skip_header=True)
test = data.TabularDataset(path=PATH+'test_data_clean.csv', format='csv', fields=data_fields, skip_header=True)

In [0]:
STATEMENT.build_vocab(train)
CONTEXT.build_vocab(train)
SUBJECT.build_vocab(train)
SPEAKER_JOB.build_vocab(train)
JUSTIFICATION.build_vocab(train)
SPEAKER.build_vocab(train)
STATE.build_vocab(train)
PARTY.build_vocab(train)

In [0]:
embedding_info = {
    'statement': {'field': STATEMENT},
    'subject': {'field': SUBJECT},
    'speaker': {'field': SPEAKER},
    'speaker_job': {'field': SPEAKER_JOB},
    'state': {'field': STATE},
    'party': {'field': PARTY},
    'context': {'field': CONTEXT},
    'justification': {'field': JUSTIFICATION}
}

In [0]:
features_word2vec = ['statement', 'subject', 'speaker_job', 'context', 'justification']
features_one_hot = ['speaker', 'state', 'party']

In [25]:
for col in features_word2vec:
  get_embedding_matrix(col)

statement     | Vocabulary size 11592	|Unknown words 1244
subject       | Vocabulary size 178	|Unknown words 4
speaker_job   | Vocabulary size 1102	|Unknown words 83
context       | Vocabulary size 2153	|Unknown words 281
justification | Vocabulary size 22266	|Unknown words 4025


In [26]:
for col in features_one_hot:
  get_one_hot_matrix(col)

speaker       | Vocabulary size 162
state         | Vocabulary size 60
party         | Vocabulary size 24


In [0]:
for col in embedding_info.keys():
  set_vectors_torch(col)

# Modeling

In [0]:
from models import LSTM_Statement, LSTMattentionFullData, CNN_Statement, LSTM_statement_metadata, LSTMFullData

In [0]:
def build_batches():
  global train_iter, val_iter, test_iter
  
  train_iter = data.BucketIterator(train,
                                batch_size=BATCH_SIZE,
                                sort_key= lambda x: len(x.statement),
                                repeat=False,
                                shuffle=True,
                                device=device)

  val_iter = data.BucketIterator(val,
                                batch_size=BATCH_SIZE,
                                sort_key= lambda x: len(x.statement),
                                repeat=False,
                                sort = True,
                                device=device)

  test_iter = data.BucketIterator(test,
                                batch_size=BATCH_SIZE,
                                sort_key= lambda x: len(x.statement),
                                repeat=False,
                                sort = True,
                                device=device)

In [0]:
def train_model(model, target_is_binary=True):
  train_logs = []
  val_logs = []
  test_logs = []

  for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train_func(train_iter, target_is_binary=target_is_binary)
    valid_loss, valid_acc = test_func(val_iter, target_is_binary=target_is_binary)
    test_loss, test_acc = test_func(test_iter, target_is_binary=target_is_binary)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')
    print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')
    print()

    train_logs += [train_loss, train_acc]
    val_logs += [valid_loss, valid_acc]
    test_logs += [test_loss, test_acc]

  return train_logs, val_logs, test_logs

In [0]:
BATCH_SIZE = 128

In [0]:
embed_size = {feature:embedding_info[feature]['embedding_matrix'].shape[1] for feature in embedding_info.keys()}

In [0]:
N_EPOCHS = 10

## Statement | LSTM

In [0]:
seed_torch()
build_batches()

model = LSTM_Statement(
    embedding_info=embedding_info, 
    embed_size=embed_size,
    output_size=6, 
    hidden_size=256, 
    fnn_layer=256*3
    ).to(device)

optimizer = optim.Adam(model.parameters(), weight_decay=1e-5)
loss_func = nn.CrossEntropyLoss().to(device)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.3)

In [123]:
log_train, log_val, log_test = train_model(model, target_is_binary=False)

Epoch: 1  | time in 0 minutes, 7 seconds
	Loss: 1.8715(train)	|	Acc: 21.3%(train)
	Loss: 2.2168(valid)	|	Acc: 19.8%(valid)
	Loss: 1.9735(test)	|	Acc: 20.8%(test)

Epoch: 2  | time in 0 minutes, 7 seconds
	Loss: 1.6780(train)	|	Acc: 29.4%(train)
	Loss: 2.0079(valid)	|	Acc: 23.0%(valid)
	Loss: 1.7776(test)	|	Acc: 23.2%(test)

Epoch: 3  | time in 0 minutes, 7 seconds
	Loss: 1.5530(train)	|	Acc: 36.6%(train)
	Loss: 2.0173(valid)	|	Acc: 24.7%(valid)
	Loss: 1.7923(test)	|	Acc: 24.6%(test)

Epoch: 4  | time in 0 minutes, 7 seconds
	Loss: 1.4787(train)	|	Acc: 40.5%(train)
	Loss: 2.0269(valid)	|	Acc: 24.4%(valid)
	Loss: 1.8001(test)	|	Acc: 24.5%(test)

Epoch: 5  | time in 0 minutes, 7 seconds
	Loss: 1.4536(train)	|	Acc: 41.9%(train)
	Loss: 2.0372(valid)	|	Acc: 24.4%(valid)
	Loss: 1.8059(test)	|	Acc: 24.9%(test)

Epoch: 6  | time in 0 minutes, 7 seconds
	Loss: 1.4441(train)	|	Acc: 42.2%(train)
	Loss: 2.0381(valid)	|	Acc: 24.2%(valid)
	Loss: 1.8077(test)	|	Acc: 24.4%(test)

Epoch: 7  | time in 0 

## Statement | CNN 

In [0]:
seed_torch()
build_batches()

model = CNN_Statement(
    embedding_info=embedding_info, 
    embed_size=embed_size, 
    output_size=6, 
    fnn_layer=512, 
    out_channels=128, 
    kernels=(3,4,5)
    ).to(device)

optimizer = optim.Adam(model.parameters(), weight_decay=1e-2)
loss_func = nn.CrossEntropyLoss().to(device)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.1)

In [119]:
log_train, log_val, log_test = train_model(model, target_is_binary=False)

Epoch: 1  | time in 0 minutes, 9 seconds
	Loss: 1.8466(train)	|	Acc: 21.0%(train)
	Loss: 2.0305(valid)	|	Acc: 20.6%(valid)
	Loss: 1.8277(test)	|	Acc: 21.0%(test)

Epoch: 2  | time in 0 minutes, 9 seconds
	Loss: 1.6207(train)	|	Acc: 34.3%(train)
	Loss: 2.0015(valid)	|	Acc: 23.2%(valid)
	Loss: 1.7857(test)	|	Acc: 23.0%(test)

Epoch: 3  | time in 0 minutes, 9 seconds
	Loss: 1.4966(train)	|	Acc: 43.5%(train)
	Loss: 2.0034(valid)	|	Acc: 22.7%(valid)
	Loss: 1.7868(test)	|	Acc: 23.0%(test)

Epoch: 4  | time in 0 minutes, 9 seconds
	Loss: 1.4758(train)	|	Acc: 44.6%(train)
	Loss: 2.0039(valid)	|	Acc: 22.9%(valid)
	Loss: 1.7871(test)	|	Acc: 22.9%(test)

Epoch: 5  | time in 0 minutes, 9 seconds
	Loss: 1.4784(train)	|	Acc: 44.4%(train)
	Loss: 2.0040(valid)	|	Acc: 22.9%(valid)
	Loss: 1.7872(test)	|	Acc: 22.9%(test)

Epoch: 6  | time in 0 minutes, 9 seconds
	Loss: 1.4764(train)	|	Acc: 43.9%(train)
	Loss: 2.0040(valid)	|	Acc: 22.9%(valid)
	Loss: 1.7872(test)	|	Acc: 22.9%(test)

Epoch: 7  | time in 0 

## Statement + Meta-data | LSTM

In [0]:
seed_torch()
build_batches()

model = LSTM_statement_metadata(
    embedding_info=embedding_info, 
    embed_size=embed_size, 
    output_size=6, 
    hidden_size=150, 
    fnn_meta_layer=512, 
    fnn_final_layer=512
    ).to(device)

optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
loss_func = nn.CrossEntropyLoss().to(device)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.3)

In [130]:
log_train, log_val, log_test = train_model(model, target_is_binary=False)

Epoch: 1  | time in 0 minutes, 5 seconds
	Loss: 1.9194(train)	|	Acc: 21.0%(train)
	Loss: 2.1114(valid)	|	Acc: 21.0%(valid)
	Loss: 1.8613(test)	|	Acc: 22.7%(test)

Epoch: 2  | time in 0 minutes, 5 seconds
	Loss: 1.7335(train)	|	Acc: 27.0%(train)
	Loss: 1.9855(valid)	|	Acc: 24.6%(valid)
	Loss: 1.7711(test)	|	Acc: 24.4%(test)

Epoch: 3  | time in 0 minutes, 5 seconds
	Loss: 1.6545(train)	|	Acc: 31.9%(train)
	Loss: 1.9740(valid)	|	Acc: 26.8%(valid)
	Loss: 1.7553(test)	|	Acc: 26.0%(test)

Epoch: 4  | time in 0 minutes, 5 seconds
	Loss: 1.6211(train)	|	Acc: 33.3%(train)
	Loss: 1.9673(valid)	|	Acc: 27.1%(valid)
	Loss: 1.7539(test)	|	Acc: 25.7%(test)

Epoch: 5  | time in 0 minutes, 5 seconds
	Loss: 1.6137(train)	|	Acc: 33.1%(train)
	Loss: 1.9683(valid)	|	Acc: 27.3%(valid)
	Loss: 1.7536(test)	|	Acc: 25.7%(test)

Epoch: 6  | time in 0 minutes, 5 seconds
	Loss: 1.6023(train)	|	Acc: 34.0%(train)
	Loss: 1.9688(valid)	|	Acc: 27.4%(valid)
	Loss: 1.7535(test)	|	Acc: 25.7%(test)

Epoch: 7  | time in 0 

## Statement + Meta-data + Justification | LSTM

In [0]:
seed_torch()
build_batches()

model = LSTMFullData(
    embedding_info=embedding_info, 
    output_size=6, 
    fnn_meta_layer=512, 
    fnn_final_layer=256, 
    embed_size=embed_size,
    hidden_size=150
    ).to(device)

optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
loss_func = nn.CrossEntropyLoss().to(device)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.3)

In [138]:
log_train, log_val, log_test = train_model(model, target_is_binary=False)

Epoch: 1  | time in 0 minutes, 14 seconds
	Loss: 1.9027(train)	|	Acc: 20.6%(train)
	Loss: 1.9811(valid)	|	Acc: 23.7%(valid)
	Loss: 1.7992(test)	|	Acc: 23.0%(test)

Epoch: 2  | time in 0 minutes, 13 seconds
	Loss: 1.7142(train)	|	Acc: 28.8%(train)
	Loss: 1.9455(valid)	|	Acc: 25.0%(valid)
	Loss: 1.7444(test)	|	Acc: 25.5%(test)

Epoch: 3  | time in 0 minutes, 14 seconds
	Loss: 1.6150(train)	|	Acc: 34.1%(train)
	Loss: 1.9463(valid)	|	Acc: 26.0%(valid)
	Loss: 1.7394(test)	|	Acc: 26.4%(test)

Epoch: 4  | time in 0 minutes, 14 seconds
	Loss: 1.5754(train)	|	Acc: 36.1%(train)
	Loss: 1.9513(valid)	|	Acc: 26.1%(valid)
	Loss: 1.7404(test)	|	Acc: 26.8%(test)

Epoch: 5  | time in 0 minutes, 14 seconds
	Loss: 1.5612(train)	|	Acc: 36.9%(train)
	Loss: 1.9537(valid)	|	Acc: 26.0%(valid)
	Loss: 1.7412(test)	|	Acc: 26.7%(test)

Epoch: 6  | time in 0 minutes, 14 seconds
	Loss: 1.5561(train)	|	Acc: 37.3%(train)
	Loss: 1.9540(valid)	|	Acc: 26.0%(valid)
	Loss: 1.7414(test)	|	Acc: 26.8%(test)

Epoch: 7  | time

## Statement + Meta-data + Justification | LSTM-Attention

In [0]:
seed_torch()
build_batches()

model = LSTMattentionFullData(
    embedding_info=embedding_info, 
    output_size=6, 
    fnn_meta_layer=512, 
    fnn_final_layer=256, 
    embed_size=embed_size,
    hidden_size=150
    ).to(device)

optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
loss_func = nn.CrossEntropyLoss().to(device)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.4)

In [111]:
log_train, log_val, log_test = train_model(model, target_is_binary=False)

Epoch: 1  | time in 0 minutes, 26 seconds
	Loss: 1.8819(train)	|	Acc: 19.9%(train)
	Loss: 1.9705(valid)	|	Acc: 23.3%(valid)
	Loss: 1.8051(test)	|	Acc: 21.3%(test)

Epoch: 2  | time in 0 minutes, 25 seconds
	Loss: 1.7417(train)	|	Acc: 23.9%(train)
	Loss: 1.9237(valid)	|	Acc: 25.2%(valid)
	Loss: 1.7674(test)	|	Acc: 25.7%(test)

Epoch: 3  | time in 0 minutes, 25 seconds
	Loss: 1.6621(train)	|	Acc: 28.7%(train)
	Loss: 1.9066(valid)	|	Acc: 26.8%(valid)
	Loss: 1.7410(test)	|	Acc: 26.4%(test)

Epoch: 4  | time in 0 minutes, 25 seconds
	Loss: 1.5995(train)	|	Acc: 32.7%(train)
	Loss: 1.9216(valid)	|	Acc: 27.2%(valid)
	Loss: 1.7492(test)	|	Acc: 26.6%(test)

Epoch: 5  | time in 0 minutes, 25 seconds
	Loss: 1.5759(train)	|	Acc: 33.7%(train)
	Loss: 1.9164(valid)	|	Acc: 26.6%(valid)
	Loss: 1.7526(test)	|	Acc: 26.5%(test)

Epoch: 6  | time in 0 minutes, 25 seconds
	Loss: 1.5574(train)	|	Acc: 34.9%(train)
	Loss: 1.9196(valid)	|	Acc: 26.7%(valid)
	Loss: 1.7572(test)	|	Acc: 26.5%(test)

Epoch: 7  | time